In [6]:
# NO QTAG ERRORS ALLOWED

"""
updated 2016-01-22 for csv mice, includes filtering
"""
import numpy as np
import pandas as pd
import regex
import os,sys
import gzip
import sqlalchemy as sqla
from functools import partial

In [1]:
EXPERIMENT = "2016-08-04-nates1"
INPUT_DIRECTORIES = ["../data/nate"]
OUTPUT_DIR = "../output"

QTAG_CSV = "../helpers/qtags_var.csv"

GTAG_MOTIF = "CGA([ACTG]{3})C([ACTG]{4})AATTCGATGG"
MCOUNT_MOTIF = "C([ACTG]{3})C([ACTG]{3})C([ACTG]{3})GCGCAACGCG"
FILE_MOTIF = "(?P<sample>.+)_(?P<sample_barcode>.+)_L(?P<lane>\d{3})_R(?P<read_number>\d)_(?P<set_number>\d{3}).fastq.gz"

In [2]:
test = '9615-01_S9_L001_R1_001.fastq.gz'

In [7]:
# used only to make regex motifs, but
# not nested to preserve qtag loading functionality if desired
def load_qtags(qtag_csv):
    try:
        qtagdf = pd.DataFrame.from_csv(qtag_csv).reset_index()
        qtagdf.rename(columns={'qtag_seq':'seq', 'qtag_num':'qid'}, inplace=True)
        qtagdf.qid = qtagdf.qid.apply(lambda x: "q%s"%str(x))
        qtagdf.seq = qtagdf.seq.str.upper()
        qtagdf.set_index('seq', inplace=True)
    except IOError as e:
        print "Unable to load qtag file, with error:", e
        sys.exit(1)
    return qtagdf

# construct regex motif dict for read search
def make_rexs():
    # load and construct qtag motif as OR list of each qtag seq (named)
    qtags = load_qtags(QTAG_CSV)
    qtag_phrases = qtags.apply(lambda x: '(?P<%s>%s)'%(x.qid, x.name) , axis=1)    
    qtag_motif = "|".join( qtag_phrases.values )
    # return compiled motifs for qtag, gtag (barcode), and molec counter, resp.
    return {'q':regex.compile(qtag_motif, flags=regex.I),
            'g':regex.compile(GTAG_MOTIF, flags=regex.I),
            'm':regex.compile(MCOUNT_MOTIF, flags=regex.I)}

In [8]:
REXS = make_rexs()


In [9]:
def testrun(db_name=None):
    all_counts = {}
    stats = {}
#     qtags = load_qtags(QTAG_CSV)
#     rexs = make_rexs(GTAG_MOTIF, MCOUNT_MOTIF, qtags)
    
#     if db_name == None:
#         db_name = 'sqlite:///%s/counts_%s.db'%(OUTPUT_DIR, EXPERIMENT)
#     else: db_name = 'sqlite:///%s/%s.db'%(OUTPUT_DIR, db_name)
        
#     engine = sqla.create_engine(db_name)
#     writer = pd.ExcelWriter('%s/filtered_%s.xlsx'%(OUTPUT_DIR,EXPERIMENT))
    iterum = 1
    directory = INPUT_DIRECTORIES[0]
    indexes = init_indexes(directory)
    for i in indexes:
#             conn = engine.connect()
        index = indexes[i]
        counts = index.init_search(rexs)
#             conn.close()
#             iterum+=1
#             all_counts[i]=counts
    engine.dispose()
    sys.stdout.write('Job complete\n')
    sys.stdout.flush()
    return all_counts, stats


In [10]:
def get_file_list(root):
    fpath_temp_a = []
    fil_temp_a = []
    # construct list of files and their infodict, as tuples:
    # (i.e. <sample>_<sample_barcode>_L<lane>_R<read_number>_<set_number>)
    for direct, sub, fil in os.walk(root):
        fpaths = np.array( [ "%s/%s"%(direct,f)  for f in fil] )
        to_append = np.array([regex.search(FILE_MOTIF,f) for f in fil ])
        fil_temp_a.append( to_append )
        fpath_temp_a.append(fpaths)
    fil_temp_b = np.concatenate(fil_temp_a)
    fpath_temp_b = np.concatenate(fpath_temp_a)
    fil_temp_c = fil_temp_b[np.nonzero(fil_temp_b)]
    fpath_temp_c = fpath_temp_b[np.nonzero(fil_temp_b)]
    files = np.array( [(fp, fil.groupdict()) for (fp, fil) in zip(fpath_temp_c, fil_temp_c)] )
    return files

In [11]:
def init_indexes(root):
    files = get_file_list(root)
    ### FIX :  files list item fmt:  (fpath, fil.str)
    indexes = dict([(f[1]['sample'],["",""]) for f in files])
    for fpath, match in files:
        if match['sample']!='Undetermined':
            # assumes 2 reads (fwd and reverse)
            indexes[match['sample']][int(match['read_number'])-1] = fpath
    if len(indexes) == 0:
        print "Empty index list. No valid files. Please check your input directory and file naming convention."
        sys.exit(1)            
    print indexes
    # convert idx entry list of files to Index object
    for idx, idx_paths in indexes.items():
        indexes[idx] = Index(idx, idx_paths)
    return indexes

In [13]:
directory = INPUT_DIRECTORIES[0]

indexes = init_indexes(directory)


{'NH003': ['../data/nate/NH003_S3_L001_R1_001.fastq.gz', '../data/nate/NH003_S3_L001_R2_001.fastq.gz'], 'NH002': ['../data/nate/NH002_S2_L001_R1_001.fastq.gz', '../data/nate/NH002_S2_L001_R2_001.fastq.gz'], 'NH001': ['../data/nate/NH001_S1_L001_R1_001.fastq.gz', '../data/nate/NH001_S1_L001_R2_001.fastq.gz'], 'NH007': ['../data/nate/NH007_S7_L001_R1_001.fastq.gz', '../data/nate/NH007_S7_L001_R2_001.fastq.gz'], 'NH006': ['../data/nate/NH006_S6_L001_R1_001.fastq.gz', '../data/nate/NH006_S6_L001_R2_001.fastq.gz'], 'NH005': ['../data/nate/NH005_S5_L001_R1_001.fastq.gz', '../data/nate/NH005_S5_L001_R2_001.fastq.gz'], 'NH004': ['../data/nate/NH004_S4_L001_R1_001.fastq.gz', '../data/nate/NH004_S4_L001_R2_001.fastq.gz'], 'NH009': ['../data/nate/NH009_S9_L001_R1_001.fastq.gz', '../data/nate/NH009_S9_L001_R2_001.fastq.gz'], 'NH008': ['../data/nate/NH008_S8_L001_R1_001.fastq.gz', '../data/nate/NH008_S8_L001_R2_001.fastq.gz'], 'NH010': ['../data/nate/NH010_S10_L001_R1_001.fastq.gz', '../data/nate/N

In [44]:
test_seq

'CCAACATGCCCTGCGCAACGCGTGCGGCCGCGGTACCCGACGGCGTGCAATTCGATGGCCTAGCTGGCATCGGTACGTGCCGCGCAACCATGTAGTAGTCCTGGAGCGTGTCCATCTGGTGTTCAAGCTTTCTTCTACAACAACCCGCTGC\n'

In [14]:
testi = indexes.values()[1]

In [45]:
testfq = gzip.open(testi.file1)

In [51]:
test_seqid = testfq.readline().strip()
test_seq = testfq.readline().strip()
test_qsid = testfq.readline().strip()
test_qs = testfq.readline().strip()


In [53]:
match = regex.search(REXS['q'],test_seq)

In [55]:
"".join(match.groups())

TypeError: sequence item 0: expected string, NoneType found

In [ ]:
# new test
class Index(object):
    READ_REF = [('q',1),('g',0),('m',0)]
    
    def __init__(self, idx, reads):
        self.idx = idx
        self.file0, self.file1 = reads
        self.tname = regex.sub('[^0-9a-zA-Z]+',"",idx)
#         except Exception, e:
#             print "Cannot open read files for %s.\nAborting with Exception: %s"%(self.idx,e)
#         else:
#             counts_dict = self.iterreads(read0, read1)
            
        # return self with modified, init Counts object outside
            #             return Counts(self.idx,counts)
    ''' offset: between end of line 4 and start of line 2 (i.e. size of line1), 
        between end of line 2 and start of line 4 (i.e. size of line3)
        can clean up a little more but this works anyway
    '''  


    def get_offset(self):
        with gzip.open(self.file0) as fq:
            first_line = fq.readline()
            pos = [0,0,0,0]
            for i in range(1,len(pos)):
                pos[i] = fq.tell()
                line = fq.readline()
            self.offset = [pos[1]-pos[0], pos[3]-pos[2]]
            
    def get_reads(fs):
        # where fs = [f0, f1]
        seqs = []
        qscores = []
        # kinda ugly but good enough; fix to accommodate classmethod
        for f in fs:
            f.seek(offset[0])
            seqs.append(f.readline().rstrip())
            f.seek(offset[1])
            qscores.append(f.readline().rstrip())
        return seqs, qscores

        ''' "key" is composed of seqs for (qtag, barcode, molec_counter)
            "scores" is composed of [barcode_score, molec_counter_score]
            (assumes that qtag score is high enough due to stringency 
            (exact match of a class seq, not a random seq))
        '''
    
    def iterreads(self):
        counts = {}
        with zip(gzip.open(self.file0), gzip.open(self.file1)) as f0, f1:
            # chunk = (seqs, qscores)
            for seqs, qscores in iter(partial(get_reads, [f0,f1]) ):
                key, scores = self.analyze_reads(seqs,qscores)
                counts.setdefault(key,[])
                counts[key].append(scores)
                
                break
        return counts
    
    def analyze_reads(self, seqs, qscores):
#         output = {'q':[],'g':[],'m':[]}

        searches = dict( [ (c, regex.search(REXS[c], seqs[r])) for c,r in READ_REF ] )
        
    
        qtag = searches['q'].lastgroup if 
        
        gtag, mcount = ('None','None')
        gscore, mscore = ('None','None')
        
#         # extract sequences and loci
#         qtag = q.lastgroup if q else 'None'
#         if g:
#             gtag = "".join(g.groups())
#             gscore = qs0[g.start():g.end()]
#         if m:
#             mcount = "".join(m.groups())
#             mscore = qs0[m.start():m.end()]
            
#         # construct key and spans tuples for handoff
#         key = (qtag,gtag,mcount)
#         scores = [gscore, mscore]
#         return key, scores

In [37]:
# # old
# class Index(object):
#     def __init__(self, idx, reads, rexs):
#         self.idx = idx
#         self.file0, self.file1 = reads[:2]
#         self.tname = regex.sub('[^0-9a-zA-Z]+',"",idx)
#         self.rexs = rexs            
    
#     def search_read(self, chunk):
#         # search for motifs in reads
#         seq0, seq1 = chunk[0]
#         qs0, qs1 = chunk[1]
#         q = regex.search(self.rexs['q'],seq1)
#         g = regex.search(self.rexs['g'],seq0)
#         m = regex.search(self.rexs['m'],seq0)
        
#         gtag, mcount = ('None','None')
#         gscore, mscore = ('None','None')
        
#         # extract sequences and loci
#         qtag = q.lastgroup if q else 'None'
#         if g:
#             gtag = "".join(g.groups())
#             gscore = qs0[g.start():g.end()]
#         if m:
#             mcount = "".join(m.groups())
#             mscore = qs0[m.start():m.end()]
            
#         # construct key and spans tuples for handoff
#         key = (qtag,gtag,mcount)
#         scores = [gscore, mscore]
#         return key, scores
    
#     def iterreads(self, read0, read1):
#         line = 2
#         counts = {}
#         # iterate through reads 
#         chunk = [(),()]
#         for r0, r1 in zip(read0, read1):
#             if line == 3:
#                 chunk[0] = (r0, r1)
#                 line = -1
#             elif line == 1:
#                 chunk[1] = (r0, r1)
#                 key, scores = self.search_read(chunk)
#                 if key in counts:
#                     counts[key].append(scores)
#                 else:
#                     counts[key] = [scores]
#             line += 1
#         return counts
    
#     def init_search(self, rexs):
#         try:
#             read0 = gzip.open(self.file0)
#             read1 = gzip.open(self.file1)
#         except Exception, e:
#             print "Cannot open read files for %s.\nAborting with Exception: %s"%(self.idx,e)
#         else:
#             counts = self.iterreads(read0, read1)
#             return Counts(self.idx,counts)

In [36]:
class Counts(object):
    def __init__(self, idx, counts):
        self.idx = idx
        self.counts = counts
    
    @staticmethod
    def convert_generator(datadict):
        i = 0
        for key in datadict:
            keyscores = datadict[key]
            q, g, m = key
            for kscore in keyscores:
                score = kscore[0]+kscore[1] if kscore[0]!='None' and kscore[1]!='None' else 'None'
                yield (i, q, g, m, score)
                i += 1
    @staticmethod
    def get_read_counts(df, q, g, m):
        qgbbool = []
        inputqgb = [q,g,m]
        tags = ['qtag','gtag','mcount']
        for i in range(len(tags)):
            b = (df[tags[i]] != 'None') if inputqgb[i] else (df[tags[i]] == 'None')
            qgbbool.append(b)
        return len(df.loc[qgbbool[0] & qgbbool[1] & qgbbool[2]])

    def convert_save_df(self):
        countsdf = pd.DataFrame(self.convert_generator(self.counts))
        countsdf.columns = ['index','qtag','gtag','mcount','score']
        self.df = countsdf
        return self
    
    def filter_reads(self):
        def classify_read(row):
            passed = 0
            minscore = np.min([ord(s) for s in row.score]) if row.score != 'None' else 0
            return 1 if minscore >= 63 else 0
        self.df['passed'] = self.df.apply(classify_read,axis=1)
        self.df = self.df.loc[self.df.qtag!='None']
        return self  
    
    def export_to_db(self, engine, if_exists='replace'):
        self.df.to_sql(self.idx, engine, if_exists=if_exists)
        return
    
    def consolidate_filter(self, writer):
        qgm_counts = pd.pivot_table(self.df.loc[self.df['passed']>0], 
                                     index=['qtag','gtag','mcount'], 
                                     values='passed', aggfunc=sum)
        if len(qgm_counts) < 1:
            self.qgcounts = pd.DataFrame()
            return self
        else:
            
            qg_counts = pd.pivot_table(pd.DataFrame(qgm_counts).reset_index(), 
                                       index=['qtag','gtag'], 
                                       values='passed', aggfunc=[sum, len])
            qg_counts.rename(columns={'len':'molecs','sum':'reads'}, inplace=True)
            qg_counts.reset_index(inplace=True)
            qg_counts.sort_values(by='molecs',ascending=False, inplace=True)
            self.qgcounts = qg_counts
            qg_counts.to_excel(writer, self.idx)
            return self
        
    def get_stats(self):
        valid = self.df.loc[(self.df.qtag!='None')&
                            (self.df.gtag!='None')&
                            (self.df.mcount!='None')]
        idxstats = {
            'total reads': len(self.df),
            'mcounts with qtag, gtag and mcount': len(valid.groupby(['qtag','gtag','mcount'])),
            'reads with qtag, gtag and mcount': len(valid),
            'reads with only no qtag': self.get_read_counts(self.df, False, True, True),
            'reads with only no gtag': self.get_read_counts(self.df, True, False, True),
            'reads with only no mcount': self.get_read_counts(self.df, True, True, False),
            'reads with only mcount': self.get_read_counts(self.df,False,False,True),
            'reads with only barcode': self.get_read_counts(self.df, False,True,False),
            'reads with only qtag': self.get_read_counts(self.df, True,False,False),
            'reads with no qtag, barcode or mcount': self.get_read_counts(self.df,False,False,False)
        }
        
        return idxstats


In [38]:
def run(db_name=None):
    all_counts = {}
    stats = {}
    qtags = load_qtags(QTAG_CSV)
    rexs = make_rexs(GTAG_MOTIF, MCOUNT_MOTIF, qtags)
    
    if db_name == None:
        db_name = 'sqlite:///%s/counts_%s.db'%(OUTPUT_DIR, EXPERIMENT)
    else: db_name = 'sqlite:///%s/%s.db'%(OUTPUT_DIR, db_name)
        
    engine = sqla.create_engine(db_name)
    writer = pd.ExcelWriter('%s/filtered_%s.xlsx'%(OUTPUT_DIR,EXPERIMENT))
    iterum = 1
    for directory in INPUT_DIRECTORIES:
        
        indexes = init_indexes(directory, rexs)
#         return    
        for i in indexes:
            conn = engine.connect()
            sys.stdout.write('Starting index %d of %d: %s\n'%(iterum, len(indexes), i))
            sys.stdout.flush()
            index = indexes[i]
            counts = index.init_search(rexs)
            sys.stdout.write('\t searched: %s\n'%i)
            sys.stdout.flush()
            try:
                counts.convert_save_df()
                sys.stdout.write('\t converted to df: %s\n'%i)
                sys.stdout.flush()                
                counts.filter_reads().consolidate_filter(writer)
                sys.stdout.write('\t filtered: %s\n'%i)
                sys.stdout.flush()
                counts.export_to_db(conn)
                sys.stdout.write('\t exported: %s\n'%i)
                sys.stdout.flush()
                stats[i] = counts.get_stats()
                sys.stdout.write('\tanalyzed statistics: %s\n'%i)
                sys.stdout.flush()
                sys.stdout.write('\t complete.\n')
                sys.stdout.flush()
            except Exception as e:
                print e
                raise
            conn.close()
            iterum+=1
            all_counts[i]=counts
    writer.save()
    engine.dispose()
    sys.stdout.write('Job complete\n')
    sys.stdout.flush()
    return all_counts, stats


In [39]:
data_counts, data_stats = run()


Starting index 1 of 51: 16314-08-Y
	 searched: 16314-08-Y
	 converted to df: 16314-08-Y
	 filtered: 16314-08-Y
	 exported: 16314-08-Y
	analyzed statistics: 16314-08-Y
	 complete.
Starting index 2 of 51: 16314-11-N
	 searched: 16314-11-N
	 converted to df: 16314-11-N
	 filtered: 16314-11-N
	 exported: 16314-11-N
	analyzed statistics: 16314-11-N
	 complete.
Starting index 3 of 51: 16614-02-Y
	 searched: 16614-02-Y
	 converted to df: 16614-02-Y
	 filtered: 16614-02-Y
	 exported: 16614-02-Y
	analyzed statistics: 16614-02-Y
	 complete.
Starting index 4 of 51: 16314-36-N
	 searched: 16314-36-N
	 converted to df: 16314-36-N
	 filtered: 16314-36-N
	 exported: 16314-36-N
	analyzed statistics: 16314-36-N
	 complete.
Starting index 5 of 51: 16314-12-N
	 searched: 16314-12-N
	 converted to df: 16314-12-N
	 filtered: 16314-12-N
	 exported: 16314-12-N
	analyzed statistics: 16314-12-N
	 complete.
Starting index 6 of 51: 16314-47-Y
	 searched: 16314-47-Y
	 converted to df: 16314-47-Y
	 filtered: 16314

In [40]:
pd.DataFrame.from_dict(data_stats).T.to_csv("%s/%s_stats.csv"%(OUTPUT_DIR,EXPERIMENT))

In [41]:
# writer = pd.ExcelWriter('filtered.xlsx')
# for idx in c:
#     cidx = Counts(c[idx].idx, c[idx].counts)
#     cidx.df = c[idx].df
#     cidx.consolidate_filter(writer)
# #     c[idx].qgcounts.to_excel(writer, idx)
#     print idx
# writer.save()